In [1]:
import math
import numpy as np
import pandas as pd
import datetime
import itertools
import random
import matplotlib.pyplot as plt
import os
os.system('wget -nc http://files.grouplens.org/datasets/movielens/ml-1m.zip')
os.system('unzip ml-1m.zip')

1

# Codi que podeu fer servir

La funció següent us permet calcular de forma molt ràpida la matriu de semblança entre usuaris. Feu-la servir enlloc de la vostra.

In [2]:
def fast_similarity_matrix(df_counts):
    """
    Retorna mitjançant càlcul matricial una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    """
    V = df_counts.values
    B = (V!=0).astype(float)
    X = np.matmul(V*V, B.T)
    XY = np.matmul(V, V.T)
    res = np.matmul(B, B.T) * ((1/(1 + np.sqrt(X - 2*XY + X.T))) - 
                                np.identity(V.shape[0])) / df_counts.shape[1]
    return res

In [10]:
data_folder = 'ml-1m'
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'generes']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')
data = pd.merge(pd.merge(ratings, users), movies)
# Reindexem els IDs dels usuaris
data['user_id'] = pd.Categorical(data['user_id']).codes
# Reindexem els IDs de les pel·lícules
data['movie_id'] = pd.Categorical(data['movie_id']).codes

In [37]:
df_counts = data.pivot_table('rating', index='user_id',columns='movie_id')
df_counts = df_counts.fillna(0)

In [38]:
df_counts.head()

movie_id,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
sim = fast_similarity_matrix(df_counts)

In [40]:
sim

array([[0.        , 0.00062961, 0.00037506, ..., 0.        , 0.0008306 ,
        0.00087497],
       [0.00062961, 0.        , 0.00066448, ..., 0.00025015, 0.00066706,
        0.00123444],
       [0.00037506, 0.00066448, 0.        , ..., 0.00040475, 0.00046871,
        0.00066864],
       ...,
       [0.        , 0.00025015, 0.00040475, ..., 0.        , 0.00058368,
        0.00040968],
       [0.0008306 , 0.00066706, 0.00046871, ..., 0.00058368, 0.        ,
        0.00144934],
       [0.00087497, 0.00123444, 0.00066864, ..., 0.00040968, 0.00144934,
        0.        ]])

# Preguntes

+ Modifica la funció `recommendAGenre` de manera que enlloc de recomanar una llista de pel·lícules a un usuari, el que faci sigui recomanar-li un gènere en concret. 

+ Per fer-ho, podem seguir aquesta recepta: Un cop tens les $n$ pel·lícules que li recomanaries segons l'algorisme que vas fer a la pràctica, mira de quin gènere són i recomanes el gènere que més es repeteixi.

+ Heu de tenir en compte que algunes pel·licules tenen diversos gèneres (per exemple `Animation|Children's|Comedy`) i que per tant els heu de separar i considerar-los tots.

In [107]:
def weighted_average(DataFrame, user, sim_mx, m):
    """
    df_counts: usuaris_semblants * pelis
    similarity: {user: similarity}
    
    Aquesta funcio calculara el score predit (Score que ha donat l'usuariB * similitud(A,B)) i despres sumara
    tots els valors per cada peli; retornara {peli_id: suma de score predit}
    """
    # la vostra solució aquí

    # Encuentra los m usuarios más similares
    similar_users = np.argsort(sim_mx[user])[-m:]
    
    # Calcula el promedio ponderado para cada película
    weighted_avg = {i: 0 for i in range(1, DataFrame.nunique()['movie_id']+1)}

    # He utilitzat aquesta variable al for, a l'hora de fer probes
    numb = int((DataFrame.nunique()['movie_id'] +1)/4)
    
    for i in range(1, DataFrame.nunique()['movie_id'] +1): 
        Simsuma = 0
        for other_user in similar_users:    
            data_user = DataFrame[DataFrame['user_id'] == other_user]
            if not data_user[data_user['movie_id'] == i]['rating'].isna().all():
                weighted_avg[i] += sim_mx[user, other_user] * data_user.loc[data_user['movie_id'] == i, 'rating'].values[0]
                Simsuma += sim_mx[user, other_user]
        if weighted_avg[i] != 0:
            weighted_avg[i] /= Simsuma
    return weighted_avg

def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """
    Retorna un dataframe de pel·licules amb els scores.
    
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_function
    :param n: nombre de pelis a recomanar
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return : pandas de pelis amb els seus scores predits
    """
    
    # la vostra solució aquí
    
    # Calcula el promedio ponderado para cada película
    weighted_avg = weighted_average(DataFrame, user, sim_mx, m)
   
    top_x_elements = sorted(weighted_avg.items(), key=lambda item: item[1], reverse=True)[:n]

    
    return top_x_elements

In [130]:
def recommendAGenre(df, user, sim, m):
    '''
    Aquesta funció ha de retornar un string amb el gènere recomanat
    '''
    
    # Codi 
    recomendacions = getRecommendationsUser(df, user, sim, 10, m)
    generes = []
    for i,x in recomendacions:
        generes_mov = df[df['movie_id'] == i]['generes'].iloc[0]
        nom =''
        for j in range(0, len(generes_mov)):
            if generes_mov[j] != "|":
                nom += generes_mov[j]
            else:
                generes.append(nom)
                nom = ''
        generes.append(nom)
    test = pd.DataFrame(generes, columns=['noms'])
    
    test2 = test.groupby('noms').count()
    
    test3 = test2.sort_values(by= "count",  ascending=False)
    return test3.iloc[0]

    

In [131]:
# Test de la funció
# Resposta correcta: Comedy

recommendAGenre(data, 3, sim, 10)

KeyError: 'count'

In [108]:
# Test de la funció
# Resposta correcta: Drama

recommendAGenre(data, 3, sim, 100)

'War'

+ Quin és el gènere que te la mitja de puntuacions més alta? Fes un codi que representi a la variable `highest_rated_genre` el nom d'aquest gènere i a la variable `highest_rating` el valor mitjà de les puntuacions. 
+ Heu de tenir en compte que algunes pel·licules tenen diversos gèneres (per exemple `Animation|Children's|Comedy`) i que per tant els heu de separar i considerar-los tots.

In [23]:
# Codi

ratings_genere = {i: 0 for i in range(1, data.nunique()['movie_id']+1)}
list_generes = []
for i in range(1, data.nunique()['movie_id'] +1):
    generes_mov = data[data['movie_id'] == i]['generes'].iloc[0]
    nom =''
    for j in range(0, len(generes_mov)):
        if generes_mov[j] != "|":
            nom += generes_mov[j]
        else:
            if nom not in list_generes:
                list_generes.append(nom)
            
            nom = ''
            





In [ ]:
# Test del resultat
# Resposta correcta: ('Film-Noir', 4.075187558184108)

highest_rated_genre, highest_rating